# As of 1/10/2020 need to figure out a way to drop all '\\' from the inspection files. Appears that we won't need to do that for crash files

In [1]:
import zipfile, re, time

In [2]:
import psycopg2 as p
import psycopg2.extras as e

In [3]:
#conn = p.connect(database = 'fleetseek', user = 'wyoung', password = 'D}av.>qPv^5hx2;^', 
#                      host = 'development-aurora-cluster.cluster-cbwvaj9awgxt.us-east-1.rds.amazonaws.com', port = 5432)
#cur = conn.cursor()

In [4]:
start_time = time.time()

In [46]:
password = 'Evergreen1'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

## Stream the text files and use copy_from to insert into a temporary table

#### 1/10/2020

In [6]:
cur.execute("""
CREATE TABLE fleetseek.inspection_temp
(
    inspection_id integer NOT NULL,
    dot_number integer,
    report_state character varying COLLATE pg_catalog."default",
    report_number character varying COLLATE pg_catalog."default",
    insp_date date,
    insp_start_time time without time zone,
    insp_end_time time without time zone,
    registration_date date,
    region character varying COLLATE pg_catalog."default",
    ci_status_code character varying COLLATE pg_catalog."default",
    safety_inspector_key character varying COLLATE pg_catalog."default",
    location character varying COLLATE pg_catalog."default",
    location_desc character varying COLLATE pg_catalog."default",
    county_code_state character varying COLLATE pg_catalog."default",
    county_code character varying COLLATE pg_catalog."default",
    insp_level_id integer,
    service_center character varying COLLATE pg_catalog."default",
    census_source_id integer,
    insp_facility character varying COLLATE pg_catalog."default",
    shipper_name character varying COLLATE pg_catalog."default",
    shipping_paper_number character varying COLLATE pg_catalog."default",
    cargo_tank character varying COLLATE pg_catalog."default",
    hazmat_placard_req character varying COLLATE pg_catalog."default",
    aspen_version_number character varying COLLATE pg_catalog."default",
    snet_version_number character varying COLLATE pg_catalog."default",
    snet_search_date date,
    alcohol_control_sub character varying COLLATE pg_catalog."default",
    drug_intrdctn_search character varying COLLATE pg_catalog."default",
    drug_intrdctn_arrests character varying COLLATE pg_catalog."default",
    size_weight_enf character varying COLLATE pg_catalog."default",
    traffic_enf character varying COLLATE pg_catalog."default",
    local_enf_jurisdiction character varying COLLATE pg_catalog."default",
    insp_confidence_level character varying COLLATE pg_catalog."default",
    pen_cen_match character varying COLLATE pg_catalog."default",
    penfield2 character varying COLLATE pg_catalog."default",
    penfield3 character varying COLLATE pg_catalog."default",
    final_status_date date,
    post_acc_ind character varying COLLATE pg_catalog."default",
    gross_comb_veh_wt character varying COLLATE pg_catalog."default",
    defect_ver character varying COLLATE pg_catalog."default",
    oos_defect_ver character varying COLLATE pg_catalog."default",
    viol_not_sent integer,
    oos_not_sent integer,
    viol_total integer,
    oos_total integer,
    driver_viol_total integer,
    driver_oos_total integer,
    vehicle_viol_total integer,
    vehicle_oos_total integer,
    
    hazmat_viol_total integer,
    hazmat_oos_total integer,
    snet_sequence_id integer,
    orig_report_state character varying COLLATE pg_catalog."default",
    orig_report_number character varying COLLATE pg_catalog."default",
    orig_report_date date,
    orig_report_time time without time zone,
    transaction_code character varying COLLATE pg_catalog."default",
    transaction_date date,
    upload_date date,
    upload_first_byte character varying COLLATE pg_catalog."default",
    upload_dot_number character varying COLLATE pg_catalog."default",
    upload_search_indicator character varying COLLATE pg_catalog."default",
    census_search_date date,
    mcmis_add_date date,
    change_by_user character varying COLLATE pg_catalog."default",
    change_date date,
    change_by_appl character varying COLLATE pg_catalog."default",
    snet_input_date date,
    source_office character varying COLLATE pg_catalog."default",
    CONSTRAINT pk_inpection_id_temp PRIMARY KEY (inspection_id)
)
""")

conn.commit()

In [7]:
#Change the filename here everytime 
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Insp_01012020_01092020\Insp_Pub_01012020_01092020.txt', 'r+') as f:
    cur.copy_from(f, 'fleetseek.inspection_temp', null = '')
    conn.commit()
    

# Compares the temporary table and table with current data

In [8]:
# ON CONFLICT ignores any rows where an inspection_id already exists
# Once the information is inserted the temporary tables get dropped
cur.execute("""
insert into fleetseek.inspection (inspection_id, dot_number,report_state, report_number, insp_date,	insp_start_time,	
                                  insp_end_time,	registration_date,	region,	ci_status_code,	safety_inspector_key,	
                                  location,	location_desc,	county_code_state,	county_code,	insp_level_id,	
                                  service_center,	census_source_id,	insp_facility,	shipper_name,	shipping_paper_number,	
                                  cargo_tank,	hazmat_placard_req,	aspen_version_number,	snet_version_number,	
                                  snet_search_date,	alcohol_control_sub,	drug_intrdctn_search,	drug_intrdctn_arrests,	
                                  size_weight_enf,	traffic_enf,	local_enf_jurisdiction,	insp_confidence_level,	
                                  pen_cen_match,	penfield2,	penfield3,	final_status_date,	post_acc_ind,	gross_comb_veh_wt,	
                                  defect_ver,	oos_defect_ver,	viol_not_sent,	oos_not_sent,	viol_total,	oos_total,	
                                  driver_viol_total,	driver_oos_total,	vehicle_viol_total,	vehicle_oos_total,	hazmat_viol_total,	
                                  hazmat_oos_total,	snet_sequence_id,	orig_report_state,	orig_report_number,	orig_report_date,	
                                  orig_report_time,	transaction_code,	transaction_date,	upload_date,	upload_first_byte,	
                                  upload_dot_number,	upload_search_indicator,	census_search_date,	mcmis_add_date,	
                                  change_by_user,	change_date,	change_by_appl,	snet_input_date,	source_office,	
                                  insp_date_start_date_time,	insp_date_end_date_time,	county_name)

select it.inspection_id, it.dot_number,it.report_state, it.report_number, it.insp_date,	it.insp_start_time,	
                                  it.insp_end_time,	it.registration_date,	it.region,	it.ci_status_code,	it.safety_inspector_key,	
                                  it.location,	it.location_desc,	it.county_code_state,	it.county_code,	it.insp_level_id,	
                                  it.service_center,	it.census_source_id,	it.insp_facility,	it.shipper_name,	it.shipping_paper_number,	
                                  it.cargo_tank,	it.hazmat_placard_req,	it.aspen_version_number,	it.snet_version_number,	
                                  it.snet_search_date,	it.alcohol_control_sub,	it.drug_intrdctn_search,	it.drug_intrdctn_arrests,	
                                  it.size_weight_enf,	it.traffic_enf,	it.local_enf_jurisdiction,	it.insp_confidence_level,	
                                  it.pen_cen_match,	it.penfield2,	it.penfield3,	it.final_status_date,	it.post_acc_ind,	it.gross_comb_veh_wt,	
                                  it.defect_ver,	it.oos_defect_ver,	it.viol_not_sent,	it.oos_not_sent,	it.viol_total,	it.oos_total,	
                                  it.driver_viol_total,	it.driver_oos_total,	it.vehicle_viol_total,	it.vehicle_oos_total,	it.hazmat_viol_total,	
                                  it.hazmat_oos_total,	it.snet_sequence_id,	it.orig_report_state,	it.orig_report_number,	it.orig_report_date,	
                                  it.orig_report_time,	it.transaction_code,	it.transaction_date,	it.upload_date,	it.upload_first_byte,	
                                  it.upload_dot_number,	it.upload_search_indicator,	it.census_search_date,	it.mcmis_add_date,	
                                  it.change_by_user,	it.change_date,	it.change_by_appl,	it.snet_input_date,	it.source_office,
                                  concat(it.insp_date, ' ', it.insp_start_time)::timestamp without time zone, concat(it.insp_date, ' ', it.insp_end_time)::timestamp without time zone,
                                  mc.countyname

from fleetseek.inspection_temp as it
join mcmiscounty as mc
on it.report_state = mc.statecode and
it.county_code::integer = mc.countycode

on conflict (inspection_id) do nothing;

DROP TABLE fleetseek.inspection_temp;

""")
conn.commit()

# 1/10/2020 Insp_Carrier_Pub test

#### Had to change the '\\' to '/' in order for copy_from to correctly read the file

In [9]:
password = 'Evergreen1'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

In [10]:
cur.execute("""
CREATE TABLE fleetseek.insp_carrier_pub_temp
(
    insp_carrier_id integer NOT NULL,
    inspection_id integer,
    insp_carrier_name character varying COLLATE pg_catalog."default",
    insp_carrier_street character varying COLLATE pg_catalog."default",
    insp_carrier_city character varying COLLATE pg_catalog."default",
    insp_carrier_state character varying COLLATE pg_catalog."default",
    insp_carrier_zip_code character varying COLLATE pg_catalog."default",
    insp_colonia character varying COLLATE pg_catalog."default",
    prefix character varying COLLATE pg_catalog."default",
    docket_number integer,
    insp_intertate character varying COLLATE pg_catalog."default",
    insp_carrier_state_id character varying COLLATE pg_catalog."default",
    CONSTRAINT fk_insp_carrier_id_temp PRIMARY KEY (insp_carrier_id)
);
""")
conn.commit()

## Change the name here everytime so it picks up the correct file

In [11]:
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Insp_01012020_01092020\Insp_Carrier_Pub_01012020_01092020.txt', 'r') as f:
    cur.copy_from(f, 'fleetseek.insp_carrier_pub_temp', null = '')
    conn.commit()

In [12]:
cur.execute(""" 
insert into fleetseek.insp_carrier_pub
select *
from fleetseek.insp_carrier_pub_temp 
on conflict(insp_carrier_id) do nothing; 

DROP TABLE fleetseek.insp_carrier_pub_temp;
""")
conn.commit()        

## Insp_study_pub

#### 1/10/2020

In [13]:
cur.execute("""
CREATE TABLE fleetseek.insp_study_pub_temp
(
    insp_study_id integer NOT NULL,
    inspection_id integer,
    study character varying COLLATE pg_catalog."default",
    seq_no character varying COLLATE pg_catalog."default",
    CONSTRAINT pk_insp_study_temp_id PRIMARY KEY (insp_study_id)
)
""")
conn.commit()

In [14]:
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Insp_01012020_01092020\Insp_Study_Pub_01012020_01092020.txt', 'r') as f:
    cur.copy_from(f, 'fleetseek.insp_study_pub_temp', null = '')
    conn.commit()

In [15]:
# Drops the temp table after inserting new data into actual table

cur.execute("""
insert into fleetseek.insp_study_pub
select *
from fleetseek.insp_study_pub_temp
on conflict(insp_study_id) do nothing;

DROP TABLE fleetseek.insp_study_pub_temp;
""")
conn.commit()

## Insp_Supp_Violation

#### 1/10/2020: this one might be slightly different due to there being no primary/foreign keys

In [16]:
# 1/10/2020 Loop back to this one because it may be a straight insert due to no primary and foreign keys
# Likely won't need to maintain this table
#cur.execute("""
#CREATE TABLE fleetseek.insp_supp_violation_temp
#(
#    insp_violation_id character varying COLLATE pg_catalog."default",
#    supp_desc character varying COLLATE pg_catalog."default"
#)
#""")

## Insp_Unit_Pub

#### 1/10/2020

In [17]:
cur.execute("""
CREATE TABLE fleetseek.insp_unit_pub_temp
(
    unit_id integer NOT NULL,
    inspection_id integer,
    unit_type_id integer,
    unit_number integer,
    unit_make character varying COLLATE pg_catalog."default",
    unit_company_number character varying COLLATE pg_catalog."default",
    unit_license character varying COLLATE pg_catalog."default",
    unit_license_state character varying COLLATE pg_catalog."default",
    unit_vehicle_id_number character varying COLLATE pg_catalog."default",
    decal_issued character varying COLLATE pg_catalog."default",
    unit_decal_number character varying COLLATE pg_catalog."default",
    CONSTRAINT pk_unit_id_temp PRIMARY KEY (unit_id)

)
""")
conn.commit()

In [18]:
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Insp_01012020_01092020\Insp_Unit_Pub_01012020_01092020.txt', 'r') as f:
    cur.copy_from(f, 'fleetseek.insp_unit_pub_temp', null = '')
    conn.commit()

In [19]:
cur.execute("""
insert into fleetseek.insp_unit_pub
select *

from fleetseek.insp_unit_pub_temp
on conflict(unit_id) do nothing;

DROP TABLE fleetseek.insp_unit_pub_temp
""")
conn.commit()

## Insp_Viol_Pub

#### 1/10/2020

In [20]:
cur.execute("""
CREATE TABLE fleetseek.insp_viol_pub_temp
(
    violation_id integer NOT NULL,
    inspection_id integer,
    viol_seq_no character varying COLLATE pg_catalog."default",
    viol_part_no character varying COLLATE pg_catalog."default",
    part_no_section character varying COLLATE pg_catalog."default",
    insp_viol_unit character varying COLLATE pg_catalog."default",
    insp_unit_id integer,
    insp_viol_gategory_id character varying COLLATE pg_catalog."default",
    out_of_service_indicator character varying COLLATE pg_catalog."default",
    defect_verification_id integer,
    citation_no character varying COLLATE pg_catalog."default",
    part_section_id integer,
    CONSTRAINT pk_violation_id_temp PRIMARY KEY (violation_id)
)
""")
conn.commit()

In [21]:
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Insp_01012020_01092020\Insp_Viol_Pub_01012020_01092020.txt', 'r') as f:
    cur.copy_from(f, 'fleetseek.insp_viol_pub_temp', null = '')
    conn.commit()

In [22]:
cur.execute("""
insert into fleetseek.insp_viol_pub
select *
from fleetseek.insp_viol_pub_temp
on conflict (violation_id) do nothing;

DROP TABLE fleetseek.insp_viol_pub_temp;
""")
conn.commit()

## Insp_ViolShip_Pub

#### 1/10/2020

In [23]:
cur.execute("""
CREATE TABLE fleetseek.insp_viol_shipper_temp
(
    insp_viol_shipper_id integer NOT NULL,
    insp_viol_id integer,
    shipper_name character varying COLLATE pg_catalog."default",
    shipper_street character varying COLLATE pg_catalog."default",
    shipper_city character varying COLLATE pg_catalog."default",
    shipper_state character varying COLLATE pg_catalog."default",
    shipper_zip_code character varying COLLATE pg_catalog."default",
    shipper_first_byte character varying COLLATE pg_catalog."default",
    shipper_dot_number character varying COLLATE pg_catalog."default",
    part_no character varying COLLATE pg_catalog."default",
    part_no_section character varying COLLATE pg_catalog."default",
    final_number varchar,
    CONSTRAINT pk_insp_viol_shipper_id_temp PRIMARY KEY (insp_viol_shipper_id)
)

""")
conn.commit()

In [24]:
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Insp_01012020_01092020\Insp_ViolShip_Pub_01012020_01092020.txt', 'r') as f:
    cur.copy_from(f, 'fleetseek.insp_viol_shipper_temp', null = '')
    conn.commit()

In [25]:
cur.execute("""
insert into fleetseek.insp_viol_shipper
select 
    insp_viol_shipper_id,
    insp_viol_id,
    shipper_name,
    shipper_street,
    shipper_city,
    shipper_state,
    shipper_zip_code,
    shipper_first_byte,
    shipper_dot_number,
    part_no,
    part_no_section
from fleetseek.insp_viol_shipper_temp
on conflict (insp_viol_shipper_id) do nothing;

DROP TABLE fleetseek.insp_viol_shipper_temp;
""")
conn.commit()

## CrashCarrier

#### 1/10/2020

In [47]:
cur.execute("""
CREATE TABLE fleetseek.crash_carrier_temp
(
    crash_carrier_id integer NOT NULL,
    crash_id integer,
    carrier_source_code integer,
    crash_carrier_name character varying COLLATE pg_catalog."default",
    crash_carrier_street character varying COLLATE pg_catalog."default",
    crash_carrier_city character varying COLLATE pg_catalog."default",
    crash_carrier_city_code character varying COLLATE pg_catalog."default",
    crash_carrier_state character varying COLLATE pg_catalog."default",
    crash_carrier_zip_code character varying COLLATE pg_catalog."default",
    crash_colonia character varying COLLATE pg_catalog."default",
    prefix character varying COLLATE pg_catalog."default",
    docket_number integer,
    crash_carrier_interstate character varying COLLATE pg_catalog."default",
    no_id_flag character varying COLLATE pg_catalog."default",
    state_number character varying COLLATE pg_catalog."default",
    state_issuing_number character varying COLLATE pg_catalog."default",
    CONSTRAINT pk_crash_carrier_id_temp PRIMARY KEY (crash_carrier_id)
)
""")
conn.commit()

In [48]:
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Crash_01012020_01092020\CrashCarrier_01012020_01092020.txt', 'r') as f:
    cur.copy_from(f, 'fleetseek.crash_carrier_temp', null = '')
    conn.commit()

In [49]:
cur.execute("""
insert into fleetseek.crash_carrier
select *
from fleetseek.crash_carrier_temp
on conflict (crash_carrier_id) do nothing;

DROP TABLE fleetseek.crash_carrier_temp;
""")
conn.commit()

## CrashEvent

#### 1/10/2020

In [50]:
cur.execute("""
CREATE TABLE fleetseek.crash_event_temp
(
    crash_event_id integer NOT NULL,
    crash_id integer,
    seq_no integer,
    event_id integer,
    event_other_desc character varying COLLATE pg_catalog."default",
    CONSTRAINT pk_crash_event_id_temp PRIMARY KEY (crash_event_id)
)
""")
conn.commit()

In [51]:
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Crash_01012020_01092020\CrashEvent_01012020_01092020.txt', 'r') as f:
    cur.copy_from(f, 'fleetseek.crash_event_temp', null = '')
    conn.commit()

In [52]:
cur.execute("""
insert into fleetseek.crash_event
select *
from fleetseek.crash_event_temp
on conflict (crash_event_id) do nothing;

DROP TABLE fleetseek.crash_event_temp;
""")
conn.commit()

## CrashMaster

#### 1/10/2020

In [53]:
password = 'Evergreen1'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

In [54]:

cur.execute("""
CREATE TABLE fleetseek.crash_master_temp
(
    crash_id integer NOT NULL,
    report_state character varying COLLATE pg_catalog."default",
    report_number character varying COLLATE pg_catalog."default",
    report_date date,
    report_time varchar,
    report_seq_no character varying COLLATE pg_catalog."default",
    dot_number integer,
    ci_status_cd character varying COLLATE pg_catalog."default",
    final_status_date date,
    location character varying COLLATE pg_catalog."default",
    city_code character varying COLLATE pg_catalog."default",
    city character varying COLLATE pg_catalog."default",
    state character varying COLLATE pg_catalog."default",
    county_code character varying COLLATE pg_catalog."default",
    truck_bus_ind character varying COLLATE pg_catalog."default",
    trafficway_id integer,
    access_control_id integer,
    road_surface_condition_id integer,
    cargo_body_type_id integer,
    gvw_rating_id integer,
    vehicle_identification_number character varying COLLATE pg_catalog."default",
    vehicle_license_number character varying COLLATE pg_catalog."default",
    vehicle_lic_state character varying COLLATE pg_catalog."default",
    vehicle_hazmat_placard character varying COLLATE pg_catalog."default",
    weather_condition_id integer,
    vehicle_configuration_id integer,
    light_condition_id integer,
    hazmat_released character varying COLLATE pg_catalog."default",
    agency character varying COLLATE pg_catalog."default",
    officer_badge character varying COLLATE pg_catalog."default",
    vehicles_in_accident character varying COLLATE pg_catalog."default",
    fatalities integer,
    injuries integer,
    tow_away character varying COLLATE pg_catalog."default",
    federal_recordable character varying COLLATE pg_catalog."default",
    state_recordable character varying COLLATE pg_catalog."default",
    snet_version_number character varying COLLATE pg_catalog."default",
    snet_sequence_id character varying COLLATE pg_catalog."default",
    transaction_code character varying COLLATE pg_catalog."default",
    transaction_date date,
    upload_first_byte character varying COLLATE pg_catalog."default",
    upload_dot_number character varying COLLATE pg_catalog."default",
    upload_search_indicator character varying COLLATE pg_catalog."default",
    upload_date date,
    census_search_date date,
    add_date date,
    change_date date,
    change_by_user character varying COLLATE pg_catalog."default",
    change_by_appl character varying COLLATE pg_catalog."default",
    snet_input_date date,
    CONSTRAINT pk_crash_id_temp PRIMARY KEY (crash_id)
)
""")
            
conn.commit()

In [55]:
with open(r'C:\Users\YoungW\OneDrive - Informa plc\Documents\Crash_01012020_01092020\CrashMaster_01012020_01092020.txt', 'r') as f:
    cur.copy_from(f, 'fleetseek.crash_master_temp', null = '')
    conn.commit()

In [56]:
# After creating the temporary table all report times with '9999' get changed to '0439'
# It appears this is how the previous process worked and I am keeping it that was as well.

cur.execute("""update fleetseek.crash_master_temp
set report_time = '0439'
where report_time = '9999';

alter table fleetseek.crash_master_temp
alter column report_time type time without time zone using report_time::time without time zone;

insert into fleetseek.crash_master
select *
from fleetseek.crash_master_temp
on conflict (crash_id) do nothing;

DROP TABLE fleetseek.crash_master_temp;
""")

conn.commit()
conn.close()

In [153]:
end_time = time.time() - start_time

In [154]:
print(end_time)

218.90823602676392
